In [10]:
from feature.clip_base_dataset import *
from feature.after_dataset import *
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [25]:
train_input_list = sorted(glob('./data/train_input/*.csv'))
train_target_list = sorted(glob('./data/train_target/*.csv'))

test_input_list = sorted(glob('./data/test_input/*.csv'))
test_target_list = sorted(glob('./data/test_target/*.csv'))

In [30]:
def group_statistics(df):  
    '''
    변수의 Case별로 DAT을 기준으로 groupby 하여 대표값 설정  
    mean, median, std, var, quantile
    '''
    df_a = pd.DataFrame()

    for i, v in enumerate(df["Case"].unique()):
        train_old = df[df['Case']==v]
        train_old = train_old.groupby(['DAT']).quantile(0.75).reset_index()
        df_a = pd.concat([df_a, train_old])
        df_a = df_a.drop(['일간누적분무량', '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량',
                          'predicted_weight_g', 'Case', 'DAT', 'obs_time'], axis=1)
    return df_a

In [31]:
train, train2 = make_dataset(train_input_list, train_target_list)
test, test2 = make_dataset(test_input_list, test_target_list)

train['obs_time'] = train2['obs_time']
train['DAT'] = train2['DAT']
test['obs_time'] = test2['obs_time']
test['DAT'] = test2['DAT']

train = time_value(train)
test = time_value(test)

train = limit_range(train)
test = limit_range(test)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

test['predicted_weight_g'] = 0

train = col_cumsum(train, "시간당분무량", "일간누적분무량")
train = col_cumsum(train, "시간당백색광량", "일간누적백색광량")
train = col_cumsum(train, "시간당적색광량", "일간누적적색광량")
train = col_cumsum(train, "시간당청색광량", "일간누적청색광량")
train = col_cumsum(train, "시간당총광량", "일간누적총광량")

In [32]:
train = group_statistics(train)
test = group_statistics(test)

In [33]:
train.to_csv('clip_train_quantile0.75.csv', index=False)
test.to_csv('clip_test_quantile0.75.csv', index=False)